In [ ]:
#testing code 
import pybullet as p
import pybullet_data
import time
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import torch
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback


# ---------------- Custom Gym Environment ----------------
class RobotEnv(gym.Env):
    def __init__(self, render=True):
        super(RobotEnv, self).__init__()
        
        self.render = render
        if self.render:
            p.connect(p.GUI)
        else:
            p.connect(p.DIRECT)

        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0, 0, -9.81)
        
        # Load environment
        self.plane = p.loadURDF("plane.urdf")
        self.robot = p.loadURDF("ryres.urdf", [0, 0, 0], useFixedBase=False,
                                flags=p.URDF_USE_SELF_COLLISION)
        
        # Wheels
        self.wheel_links = [9, 10, 12]   # tyre links
        for link in self.wheel_links:
            p.changeDynamics(self.robot, link,
                     lateralFriction=0.2,
                     spinningFriction=0.5,
                     rollingFriction=0.01)
            
        self.wheel_joints = [7, 8]   # drive joints
        self.max_force = 50.0
        self.max_speed = 20.0
        
        # ✅ Action space: [forward_speed (0..1), turn_rate (-1..1)]
        # self.action_space = spaces.Box(low=np.array([0.0, -1.0]), 
        #                                high=np.array([2.0, 1.0]),
        #                                dtype=np.float32)
        self.action_space = spaces.Discrete(3)  # 0=forward, 1=left, 2=right
        
        # Observation space
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(4,), dtype=np.float32)
        
        # Episode settings
        self.max_steps = 10000
        self.current_step = 0
        self.goal_threshold = 0.05
        
        # Visual marker for goal
        self.goal_vis_id = None
        # ✅ Counters
        self.prev_dist = None 
        self.forward_count = 0
        self.left_turn_count = 0
        self.right_turn_count=0
        self.total_steps = 0

    def reset(self, seed=None, options=None):
        self.current_step = 0
        self.forward_count = 0
        self.left_turn_count = 0
        self.right_turn_count=0
        self.total_steps = 0
        # print("🔄 Counters reset.")
        # self.current_step = 0
        # Reset robot
        p.resetBasePositionAndOrientation(self.robot, [0, 0, 0.125], [0, 0, 0, 1])
        

        for i in range(p.getNumJoints(self.robot)):
            p.resetJointState(self.robot, i, 0)

        # Disable drive wheel motors
        for j in [7, 8]:  
            p.setJointMotorControl2(self.robot, j, p.VELOCITY_CONTROL,
                            targetVelocity=0, force=0)

        # Lock other joints
        for i in range(p.getNumJoints(self.robot)):
            if i not in [7, 8]:
                p.setJointMotorControl2(self.robot, i, p.VELOCITY_CONTROL,
                                targetVelocity=0, force=self.max_force)

        # Random goal
        self.goal = np.random.uniform(-3.5, 3.5, size=2)
        goal_pos = [self.goal[0], self.goal[1], 0.1]

        if self.goal_vis_id is not None:
            p.removeBody(self.goal_vis_id)

        vis_shape = p.createVisualShape(p.GEOM_SPHERE, radius=0.1, rgbaColor=[1, 0, 0, 1])
        col_shape = p.createCollisionShape(p.GEOM_SPHERE, radius=0.001)
        self.goal_vis_id = p.createMultiBody(baseMass=0,
                                            baseVisualShapeIndex=vis_shape,
                                            baseCollisionShapeIndex=col_shape,
                                            basePosition=goal_pos)
        
        gripper_pos, _ = p.getLinkState(self.robot, 4)[:2]
        self.prev_dist = np.linalg.norm(np.array(gripper_pos[:2]) - self.goal)
        _, base_orn = p.getBasePositionAndOrientation(self.robot)
        yaw = p.getEulerFromQuaternion(base_orn)[2] - np.pi/2
        gripper_pos, _ = p.getLinkState(self.robot, 4)[:2]
        gripper_xy = np.array(gripper_pos[:2])
        goal_angle = np.arctan2(self.goal[1]-gripper_xy[1], self.goal[0]-gripper_xy[0])
        self.prev_heading_error = abs((goal_angle - yaw + np.pi) % (2*np.pi) - np.pi)


        return self._get_obs(), {}
    ##################################################################
    def step(self, action):
    # Map discrete actions to wheel speeds
        if action == 0:  # Forward
            left_vel = self.max_speed
            right_vel = self.max_speed
            self.forward_count += 1

        elif action == 1:  # Turn left
            left_vel = -0.5 * self.max_speed
            right_vel =  0.5 * self.max_speed
            self.left_turn_count += 1

        elif action == 2:  # Turn right
            left_vel =  0.5 * self.max_speed
            right_vel = -0.5 * self.max_speed
            self.right_turn_count += 1

    # Apply motor control
        p.setJointMotorControl2(self.robot, 7, p.VELOCITY_CONTROL,
                                targetVelocity=left_vel, force=self.max_force)
        p.setJointMotorControl2(self.robot, 8, p.VELOCITY_CONTROL,
                                targetVelocity=right_vel, force=self.max_force)

        p.stepSimulation()
        self.current_step += 1
        self.total_steps += 1

    # --- rest of your reward, obs, done logic stays same ---
        obs = self._get_obs()
        gripper_pos, _ = p.getLinkState(self.robot, 4)[:2]
        gripper_xy = np.array(gripper_pos[:2])
        dist_to_goal = np.linalg.norm(gripper_xy - self.goal)

        progress = self.prev_dist - dist_to_goal
        self.prev_dist = dist_to_goal

        _, base_orn = p.getBasePositionAndOrientation(self.robot)
        yaw = p.getEulerFromQuaternion(base_orn)[2] - np.pi/2   # align with -Y front
        goal_angle = np.arctan2(self.goal[1]-gripper_xy[1], self.goal[0]-gripper_xy[0])
        heading_error = abs((goal_angle - yaw + np.pi) % (2*np.pi) - np.pi)
        

        # reward = 0.0
        # reward += progress * 10.0
        # reward += np.cos(heading_error) * 0.5 
        # reward -= 0.01
        # --- Reward shaping ---
        reward = 0.0

        # Distance progress reward (main driver)
        reward += progress * 10.0  

        # Heading alignment reward (encourages facing goal)
        reward += np.cos(heading_error) * 0.5  

        # Penalize step cost slightly
        reward -= 0.045

        # ✅ Turning purposefully: only reward if it reduces heading error
        if action in [1, 2]:  # turn left or right
            if heading_error < self.prev_heading_error:  
                reward += 0.3   # good turn (aligning to goal)
            else:
                reward -= 0.3   # bad turn (misaligned more)

    #Save heading error for next step comparison
        self.prev_heading_error = heading_error

        done, truncated = False, False
        if dist_to_goal < self.goal_threshold:
            reward += 100
            done = True
            print(f"✅ Goal reached!")
            print(f"   Goal Position   = {self.goal}")
            print(f"   Gripper Position= {gripper_xy}")
            print(f"📊 Summary: Forward={self.forward_count}, Left-Turn={self.left_turn_count}, Right-Turn={self.right_turn_count} Steps={self.total_steps}")
        elif self.current_step >= self.max_steps:
            reward -= 50
            truncated = True
            print(f"⏳ Timeout. Reward={reward:.2f}")
            print(f"   Goal Position   = {self.goal}")
            print(f"   Gripper Position= {gripper_xy}")
            print(f"📊 Summary: Forward={self.forward_count}, Left-Turn={self.left_turn_count}, Right-Turn={self.right_turn_count} Steps={self.total_steps}")
    
        return obs, reward, done, truncated, {}

    def _get_obs(self):
        gripper_pos, _ = p.getLinkState(self.robot, 4)[:2]
        gripper_xy = np.array(gripper_pos[:2])
        _, base_orn = p.getBasePositionAndOrientation(self.robot)
        yaw = p.getEulerFromQuaternion(base_orn)[2]

        goal_vec = self.goal - gripper_xy
        dist = np.linalg.norm(goal_vec)
        angle = np.arctan2(goal_vec[1], goal_vec[0]) - yaw
        angle = (angle + np.pi) % (2 * np.pi) - np.pi
        
        return np.array([gripper_xy[0], gripper_xy[1], dist, angle], dtype=np.float32)

    def close(self):
        p.disconnect()


# ---------------- Callback for tracking episodes ----------------
class EpisodeTracker(BaseCallback):
    def __init__(self):
        super().__init__()
        self.episode_rewards = []
        self.episode_num = 0
        self.episode_reward = 0

    def _on_step(self) -> bool:
        reward = self.locals["rewards"][0]
        done = self.locals["dones"][0]
        self.episode_reward += reward

        if done:
            self.episode_num += 1
            print(f"🎯 Episode {self.episode_num} finished with reward {self.episode_reward:.2f}")
            self.episode_rewards.append(self.episode_reward)
            self.episode_reward = 0
        return True


# ---------------- Training ----------------
def train():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")
    
    env = DummyVecEnv([lambda: RobotEnv(render=False)])
    callback = EpisodeTracker()
    
    model = PPO(
        "MlpPolicy",
        env,
        learning_rate=3e-4,
        n_steps=512,
        batch_size=64,
        n_epochs=10,
        gamma=0.99,
        gae_lambda=0.95,
        clip_range=0.2,
        ent_coef=0.0,
        vf_coef=0.5,
        max_grad_norm=0.5,
        verbose=0,
        device=device,
    )
    
    model.learn(total_timesteps=5000_000, callback=callback)
    model.save("stage1_nav.zip")
    print("✅ Model saved as stage1_nav.zip")


# ---------------- Testing ----------------
def test():
    env = RobotEnv(render=True)
    model = PPO.load("stage1_nav.zip")
    
    obs, _ = env.reset()
    done, truncated = False, False
    total_reward = 0
    while not (done or truncated):
        action, _ = model.predict(obs)
        obs, reward, done, truncated, _ = env.step(action)
        total_reward += reward
        time.sleep(0.01)
    print(f"Episode finished with total reward = {total_reward:.2f}")
    env.close()


if __name__ == "__main__":
    train()
    # test()


Using device: cpu
⏳ Timeout. Reward=-49.67
   Goal Position   = [-2.79648935  2.93000542]
   Gripper Position= [-0.70411123 -1.57325237]
📊 Summary: Forward=1899, Left-Turn=1310, Right-Turn=2791 Steps=6000
🎯 Episode 1 finished with reward -850.39
⏳ Timeout. Reward=-50.85
   Goal Position   = [-0.7489157  -2.72992241]
   Gripper Position= [-4.54595197  1.08939197]
📊 Summary: Forward=2280, Left-Turn=2183, Right-Turn=1537 Steps=6000
🎯 Episode 2 finished with reward -1243.70
⏳ Timeout. Reward=-49.24
   Goal Position   = [-0.28760654  3.13893256]
   Gripper Position= [ 2.34207708 -0.91729941]
📊 Summary: Forward=1657, Left-Turn=2849, Right-Turn=1494 Steps=6000
🎯 Episode 3 finished with reward -724.78
⏳ Timeout. Reward=-49.47
   Goal Position   = [-2.2219468   1.05667523]
   Gripper Position= [ 0.39432794 -0.50175382]
📊 Summary: Forward=814, Left-Turn=4524, Right-Turn=662 Steps=6000
🎯 Episode 4 finished with reward -851.97
⏳ Timeout. Reward=-50.00
   Goal Position   = [ 2.53408024 -3.79447258]

In [ ]:
#testing code 
import pybullet as p
import pybullet_data
import time
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import torch
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback


# ---------------- Custom Gym Environment ----------------
class RobotEnv(gym.Env):
    def __init__(self, render=True):
        super(RobotEnv, self).__init__()
        
        self.render = render
        if self.render:
            p.connect(p.GUI)
        else:
            p.connect(p.DIRECT)

        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0, 0, -9.81)
        
        # Load environment
        self.plane = p.loadURDF("plane.urdf")
        self.robot = p.loadURDF("ryres.urdf", [0, 0, 0], useFixedBase=False,
                                flags=p.URDF_USE_SELF_COLLISION)
        
        # Wheels
        self.wheel_links = [9, 10, 12]   # tyre links
        for link in self.wheel_links:
            p.changeDynamics(self.robot, link,
                     lateralFriction=0.2,
                     spinningFriction=0.5,
                     rollingFriction=0.01)
            
        self.wheel_joints = [7, 8]   # drive joints
        self.max_force = 50.0
        self.max_speed = 20.0
        
        # ✅ Action space: [forward_speed (0..1), turn_rate (-1..1)]
        # self.action_space = spaces.Box(low=np.array([0.0, -1.0]), 
        #                                high=np.array([2.0, 1.0]),
        #                                dtype=np.float32)
        self.action_space = spaces.Discrete(3)  # 0=forward, 1=left, 2=right
        
        # Observation space
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(4,), dtype=np.float32)
        
        # Episode settings
        self.max_steps = 10000
        self.current_step = 0
        self.goal_threshold = 0.05
        
        # Visual marker for goal
        self.goal_vis_id = None
        # ✅ Counters
        self.prev_dist = None 
        self.forward_count = 0
        self.left_turn_count = 0
        self.right_turn_count=0
        self.total_steps = 0

    def reset(self, seed=None, options=None):
        self.current_step = 0
        self.forward_count = 0
        self.left_turn_count = 0
        self.right_turn_count=0
        self.total_steps = 0
        # print("🔄 Counters reset.")
        # self.current_step = 0
        # Reset robot
        p.resetBasePositionAndOrientation(self.robot, [0, 0, 0.125], [0, 0, 0, 1])
        

        for i in range(p.getNumJoints(self.robot)):
            p.resetJointState(self.robot, i, 0)

        # Disable drive wheel motors
        for j in [7, 8]:  
            p.setJointMotorControl2(self.robot, j, p.VELOCITY_CONTROL,
                            targetVelocity=0, force=0)

        # Lock other joints
        for i in range(p.getNumJoints(self.robot)):
            if i not in [7, 8]:
                p.setJointMotorControl2(self.robot, i, p.VELOCITY_CONTROL,
                                targetVelocity=0, force=self.max_force)

        # Random goal
        self.goal = np.random.uniform(-3.5, 3.5, size=2)
        goal_pos = [self.goal[0], self.goal[1], 0.1]

        if self.goal_vis_id is not None:
            p.removeBody(self.goal_vis_id)

        vis_shape = p.createVisualShape(p.GEOM_SPHERE, radius=0.1, rgbaColor=[1, 0, 0, 1])
        col_shape = p.createCollisionShape(p.GEOM_SPHERE, radius=0.001)
        self.goal_vis_id = p.createMultiBody(baseMass=0,
                                            baseVisualShapeIndex=vis_shape,
                                            baseCollisionShapeIndex=col_shape,
                                            basePosition=goal_pos)
        
        gripper_pos, _ = p.getLinkState(self.robot, 4)[:2]
        self.prev_dist = np.linalg.norm(np.array(gripper_pos[:2]) - self.goal)
        _, base_orn = p.getBasePositionAndOrientation(self.robot)
        yaw = p.getEulerFromQuaternion(base_orn)[2] - np.pi/2
        gripper_pos, _ = p.getLinkState(self.robot, 4)[:2]
        gripper_xy = np.array(gripper_pos[:2])
        goal_angle = np.arctan2(self.goal[1]-gripper_xy[1], self.goal[0]-gripper_xy[0])
        self.prev_heading_error = abs((goal_angle - yaw + np.pi) % (2*np.pi) - np.pi)


        return self._get_obs(), {}
    ##################################################################
    def step(self, action):
    # Map discrete actions to wheel speeds
        if action == 0:  # Forward
            left_vel = self.max_speed
            right_vel = self.max_speed
            self.forward_count += 1

        elif action == 1:  # Turn left
            left_vel = -0.5 * self.max_speed
            right_vel =  0.5 * self.max_speed
            self.left_turn_count += 1

        elif action == 2:  # Turn right
            left_vel =  0.5 * self.max_speed
            right_vel = -0.5 * self.max_speed
            self.right_turn_count += 1

    # Apply motor control
        p.setJointMotorControl2(self.robot, 7, p.VELOCITY_CONTROL,
                                targetVelocity=left_vel, force=self.max_force)
        p.setJointMotorControl2(self.robot, 8, p.VELOCITY_CONTROL,
                                targetVelocity=right_vel, force=self.max_force)

        p.stepSimulation()
        self.current_step += 1
        self.total_steps += 1

    # --- rest of your reward, obs, done logic stays same ---
        obs = self._get_obs()
        gripper_pos, _ = p.getLinkState(self.robot, 4)[:2]
        gripper_xy = np.array(gripper_pos[:2])
        dist_to_goal = np.linalg.norm(gripper_xy - self.goal)

        progress = self.prev_dist - dist_to_goal
        self.prev_dist = dist_to_goal

        _, base_orn = p.getBasePositionAndOrientation(self.robot)
        yaw = p.getEulerFromQuaternion(base_orn)[2] - np.pi/2   # align with -Y front
        goal_angle = np.arctan2(self.goal[1]-gripper_xy[1], self.goal[0]-gripper_xy[0])
        heading_error = abs((goal_angle - yaw + np.pi) % (2*np.pi) - np.pi)
        

        # reward = 0.0
        # reward += progress * 10.0
        # reward += np.cos(heading_error) * 0.5 
        # reward -= 0.01
        # --- Reward shaping ---
        reward = 0.0

        # Distance progress reward (main driver)
        reward += progress * 10.0  

        # Heading alignment reward (encourages facing goal)
        reward += np.cos(heading_error) * 0.5  

        # Penalize step cost slightly
        reward -= 0.045

        # ✅ Turning purposefully: only reward if it reduces heading error
        if action in [1, 2]:  # turn left or right
            if heading_error < self.prev_heading_error:  
                reward += 0.3   # good turn (aligning to goal)
            else:
                reward -= 0.3   # bad turn (misaligned more)

    #Save heading error for next step comparison
        self.prev_heading_error = heading_error

        done, truncated = False, False
        if dist_to_goal < self.goal_threshold:
            reward += 100
            done = True
            print(f"✅ Goal reached!")
            print(f"   Goal Position   = {self.goal}")
            print(f"   Gripper Position= {gripper_xy}")
            print(f"📊 Summary: Forward={self.forward_count}, Left-Turn={self.left_turn_count}, Right-Turn={self.right_turn_count} Steps={self.total_steps}")
        elif self.current_step >= self.max_steps:
            reward -= 50
            truncated = True
            print(f"⏳ Timeout. Reward={reward:.2f}")
            print(f"   Goal Position   = {self.goal}")
            print(f"   Gripper Position= {gripper_xy}")
            print(f"📊 Summary: Forward={self.forward_count}, Left-Turn={self.left_turn_count}, Right-Turn={self.right_turn_count} Steps={self.total_steps}")
    
        return obs, reward, done, truncated, {}

    def _get_obs(self):
        gripper_pos, _ = p.getLinkState(self.robot, 4)[:2]
        gripper_xy = np.array(gripper_pos[:2])
        _, base_orn = p.getBasePositionAndOrientation(self.robot)
        yaw = p.getEulerFromQuaternion(base_orn)[2]

        goal_vec = self.goal - gripper_xy
        dist = np.linalg.norm(goal_vec)
        angle = np.arctan2(goal_vec[1], goal_vec[0]) - yaw
        angle = (angle + np.pi) % (2 * np.pi) - np.pi
        
        return np.array([gripper_xy[0], gripper_xy[1], dist, angle], dtype=np.float32)

    def close(self):
        p.disconnect()


# ---------------- Callback for tracking episodes ----------------
class EpisodeTracker(BaseCallback):
    def __init__(self):
        super().__init__()
        self.episode_rewards = []
        self.episode_num = 0
        self.episode_reward = 0

    def _on_step(self) -> bool:
        reward = self.locals["rewards"][0]
        done = self.locals["dones"][0]
        self.episode_reward += reward

        if done:
            self.episode_num += 1
            print(f"🎯 Episode {self.episode_num} finished with reward {self.episode_reward:.2f}")
            self.episode_rewards.append(self.episode_reward)
            self.episode_reward = 0
        return True


# ---------------- Training ----------------
def train():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")
    
    env = DummyVecEnv([lambda: RobotEnv(render=False)])
    callback = EpisodeTracker()
    
    model = PPO(
        "MlpPolicy",
        env,
        learning_rate=3e-4,
        n_steps=512,
        batch_size=64,
        n_epochs=10,
        gamma=0.99,
        gae_lambda=0.95,
        clip_range=0.2,
        ent_coef=0.0,
        vf_coef=0.5,
        max_grad_norm=0.5,
        verbose=0,
        device=device,
    )
    
    model.learn(total_timesteps=5000_000, callback=callback)
    model.save("stage1_nav.zip")
    print("✅ Model saved as stage1_nav.zip")


# ---------------- Testing ----------------
def test():
    env = RobotEnv(render=True)
    model = PPO.load("stage1_nav.zip")
    
    obs, _ = env.reset()
    done, truncated = False, False
    total_reward = 0
    while not (done or truncated):
        action, _ = model.predict(obs)
        obs, reward, done, truncated, _ = env.step(action)
        total_reward += reward
        time.sleep(0.01)
    print(f"Episode finished with total reward = {total_reward:.2f}")
    env.close()


if __name__ == "__main__":
    # train()
    test()


✅ Goal reached!
   Goal Position   = [-0.9938412   0.02388495]
   Gripper Position= [-0.94644331  0.03758538]
📊 Summary: Forward=318, Left-Turn=770, Right-Turn=0 Steps=1088
Episode finished with total reward = 260.78
